In [1]:
from SeqGAN.model import *

/home/yuntae_kang/.pyenv/versions/3.12.5/envs/imb_data/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [3]:
ACTIONS = ['[PAD]', '[SOS]', 'Logon', 'Logoff', 'Connect', 'Disconnect']
vocab = Vocabulary(ACTIONS)

In [5]:
BATCH_SIZE = 8
MAX_LENGTH = 15
DEVICE = torch.device('cuda')

batch_samples = []
for _ in range(BATCH_SIZE):
    length = torch.randint(1, MAX_LENGTH, (1,))[0]
    sample = torch.randint(2, 6, size=(length,))
    sample = torch.cat([torch.tensor([1]), sample], dim=0)
    batch_samples.append(sample)

batch_samples = torch.nn.utils.rnn.pad_sequence(batch_samples, batch_first=True).to(DEVICE)

In [6]:
generator = Generator(vocab, 4, 3).to(DEVICE)
discriminator = Discriminator(vocab, 4, [2], [2], .25).to(DEVICE)
pg_criterion = PGLoss()
gen_optimizer = torch.optim.Adam(generator.parameters(), lr=0.01, betas=[0.9, 0.99])

In [7]:
pad_id = vocab.action_dict['[PAD]']
criterion = nn.NLLLoss(ignore_index=pad_id)

target = torch.cat([batch_samples[:,1:], torch.full(size=(BATCH_SIZE, 1), fill_value=pad_id, device=batch_samples.device)], dim=1)

log_probs = generator(batch_samples)
gen_loss = criterion(log_probs.view(-1, len(vocab)), target.view(-1))

In [8]:
gen_optimizer.zero_grad()
gen_loss.backward()
gen_optimizer.step()

In [9]:
source = torch.ones(size=(BATCH_SIZE, 1)).long().to(DEVICE)
generated_samples = generator.generate(source=source, target_len=20)

In [10]:
log_P = generator(generated_samples)

In [11]:
rewards = action_value_function(generated_samples, generator, discriminator, 10)

In [12]:
log_probs = log_P[:, :-1].gather(dim=-1, index=generated_samples[:, 1:].unsqueeze(-1)).squeeze()

In [14]:
pg_loss = pg_criterion(log_probs, rewards)

In [16]:
pg_loss.backward()

In [94]:
loss = log_probs * rewards

In [95]:
lengths = loss.ne(0).sum(dim=1).unsqueeze(dim=1) + 1e-9

In [100]:
-(loss / lengths).sum(dim=1).mean()

tensor(1.0193, device='cuda:0', grad_fn=<NegBackward0>)

In [15]:
rewards

tensor([[0.6142, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6201, 0.6349, 0.6379, 0.6406, 0.6406, 0.6411],
        [0.6286, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6162, 0.6481, 0.6387, 0.6367, 0.0000, 0.0000],
        [0.6180, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6297, 0.6332, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6203, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]], device='cuda:0')

In [33]:
generated_samples

tensor([[1, 3, 3, 2, 2, 4, 4, 4, 0, 0],
        [1, 4, 4, 4, 3, 4, 4, 4, 5, 3],
        [1, 5, 4, 5, 4, 5, 0, 0, 0, 0],
        [1, 5, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 5, 4, 5, 4, 0, 0, 0, 0, 0],
        [1, 5, 4, 5, 0, 0, 0, 0, 0, 0],
        [1, 4, 4, 0, 0, 0, 0, 0, 0, 0],
        [1, 4, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [ ]:
log_P.index

In [32]:
log_P[:, :-1, :].shape

torch.Size([8, 9, 6])

In [26]:
rewards.shape

torch.Size([8, 9])

In [28]:
~(generated_samples.eq(0)[:, 1:])

tensor([[ True,  True,  True, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False, False, False,
         

In [27]:
rewards * ~(generated_samples.eq(0)[:, 1:])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       device='cuda:0')

In [19]:
source

tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]], device='cuda:0')

In [37]:
a.sum(dim=1).mean()

tensor(0.6810)

In [31]:
a/a.ne(0).sum(dim=1).unsqueeze(1)

tensor([[-0.5903, -0.0637, -0.0097,  0.0000,  0.0000],
        [-0.0180, -0.0060,  0.2568, -0.1708,  0.3266],
        [ 0.4415,  0.0694,  0.0000,  0.0000,  0.0000],
        [-0.1361, -0.0011,  0.1317,  0.1980, -0.2454],
        [-0.1076, -0.0454, -0.1535,  0.0218,  0.1662],
        [-0.2480, -0.5456, -0.1236,  0.4397,  0.5175],
        [-0.0112,  0.0340,  0.4097, -0.0121,  0.3133],
        [-0.1524,  0.2566,  0.0459,  0.0384,  0.1617],
        [-0.0242, -0.0717,  0.0888,  0.1620,  0.0059],
        [-0.0615,  0.0739, -0.0098, -0.0149,  0.0663]])

In [62]:
actions = {'[PAD]', '[SOS]', 'Logon', 'Logoff', 'Connect', 'Disconnect'}
vocabs = Vocabulary(actions)

generator = Generator(vocabs, 5, 4)

In [58]:
source = torch.full(size=(5, 1), fill_value=generator.embedding.sos_id)

In [83]:
gen = generator.generate(source, 10)

In [84]:
gen

tensor([[4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [4, 5, 4, 3, 0, 0, 0, 0, 0, 0],
        [2, 4, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 5, 2, 4, 0, 0, 0, 0, 0, 0],
        [4, 3, 4, 0, 0, 0, 0, 0, 0, 0]])

In [30]:
import torch

/home/yuntae_kang/.pyenv/versions/3.12.5/envs/imb_data/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [72]:
x = torch.cat([torch.randint(1, 5, size=(10, 3)), torch.zeros(10, 1).long()], dim=1)
print(x)
pad_mask = x.eq(0)

x = torch.cat([torch.full(size=(10, 1), fill_value=1, device=x.device), x], dim=1)
print(x)
x = x[:, :-1]
print(x[~pad_mask])


tensor([[3, 3, 3, 0],
        [3, 4, 3, 0],
        [2, 1, 1, 0],
        [1, 3, 1, 0],
        [4, 2, 4, 0],
        [1, 4, 1, 0],
        [1, 2, 4, 0],
        [1, 2, 3, 0],
        [1, 4, 1, 0],
        [1, 3, 2, 0]])
tensor([[1, 3, 3, 3, 0],
        [1, 3, 4, 3, 0],
        [1, 2, 1, 1, 0],
        [1, 1, 3, 1, 0],
        [1, 4, 2, 4, 0],
        [1, 1, 4, 1, 0],
        [1, 1, 2, 4, 0],
        [1, 1, 2, 3, 0],
        [1, 1, 4, 1, 0],
        [1, 1, 3, 2, 0]])
tensor([1, 3, 3, 1, 3, 4, 1, 2, 1, 1, 1, 3, 1, 4, 2, 1, 1, 4, 1, 1, 2, 1, 1, 2,
        1, 1, 4, 1, 1, 3])


In [42]:
torch.sort(x, descending=True)

torch.return_types.sort(
values=tensor([[4, 3, 2],
        [4, 3, 1],
        [2, 1, 0],
        [4, 1, 0],
        [4, 2, 0],
        [3, 0, 0],
        [4, 3, 1],
        [2, 2, 1],
        [4, 2, 2],
        [2, 2, 0]]),
indices=tensor([[0, 2, 1],
        [2, 1, 0],
        [1, 2, 0],
        [2, 0, 1],
        [0, 1, 2],
        [0, 1, 2],
        [2, 0, 1],
        [0, 2, 1],
        [1, 0, 2],
        [1, 2, 0]]))

In [34]:
torch.full((2, 3), 2)

tensor([[2, 2, 2],
        [2, 2, 2]])

In [25]:
def generate_action_dict(actions: set):
    """
    Args:
        actions: set of action-ids (including special tokens e.g. [SOS], [PAD])
    """
    actions = actions.copy()
    action_dict = {}
    
    if '[PAD]' in actions:
        action_dict[0] = '[PAD]'
        actions.remove('[PAD]')
    if '[SOS]' in actions:
        action_dict[1] = '[SOS]'
        actions.remove('[SOS]')

    for idx, action_id in enumerate(actions):
        action_dict[idx+2] = action_id

    return action_dict

In [26]:
action_dict = generate_action_dict(a)

In [29]:
action_dict

{0: '[PAD]', 1: '[SOS]', 2: 'Logoff', 3: 'Logon'}

In [22]:
a

{1, 2, 3, '[PAD]', '[SOS]'}

In [20]:
b.remove(1)

In [21]:
a

{1, 2, 3, '[PAD]', '[SOS]'}

In [12]:
a

{2, 3, '[PAD]', '[SOS]'}

In [8]:
a.discard(4)

In [165]:
from .SeqGAN_Pytorch import model

ImportError: attempted relative import with no known parent package

In [13]:
import torch
from torch import nn

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [2]:
a = torch.tensor([[1,2,3,0,0], [1,2,0,0,0],[1,2,3,4,5]])

In [28]:
embedding = nn.Embedding(num_embeddings=6, embedding_dim=3, padding_idx=0)
embedded = embedding(a)
lengths = a.ne(0).sum(axis=1).cpu()
packed_embedded = pack_padded_sequence(embedded, lengths=lengths, batch_first=True, enforce_sorted=False)

In [49]:
lstm = nn.LSTM(input_size=3, hidden_size=4, batch_first=True)
packed_output, _ = lstm(packed_embedded)

In [50]:
output, _ = pad_packed_sequence(packed_output, batch_first=True)

In [56]:
pad_mask = a.eq(0)

In [52]:
output

tensor([[[0.0492, 0.1254, 0.2025, 0.0362],
         [0.0906, 0.1724, 0.3160, 0.0057],
         [0.2148, 0.1673, 0.4380, 0.0471],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0492, 0.1254, 0.2025, 0.0362],
         [0.0906, 0.1724, 0.3160, 0.0057],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0492, 0.1254, 0.2025, 0.0362],
         [0.0906, 0.1724, 0.3160, 0.0057],
         [0.2148, 0.1673, 0.4380, 0.0471],
         [0.2691, 0.1404, 0.3402, 0.1812],
         [0.3480, 0.0616, 0.1591, 0.2358]]], grad_fn=<IndexSelectBackward0>)

In [58]:
linear = nn.Linear(4, 5)

In [60]:
linear(output)

tensor([[[ 0.2169, -0.2792,  0.1324,  0.1725, -0.3363],
         [ 0.2146, -0.2847,  0.0831,  0.1790, -0.3274],
         [ 0.1729, -0.2822,  0.0625,  0.1681, -0.3329],
         [ 0.2081, -0.3069,  0.1994,  0.1640, -0.3619],
         [ 0.2081, -0.3069,  0.1994,  0.1640, -0.3619]],

        [[ 0.2169, -0.2792,  0.1324,  0.1725, -0.3363],
         [ 0.2146, -0.2847,  0.0831,  0.1790, -0.3274],
         [ 0.2081, -0.3069,  0.1994,  0.1640, -0.3619],
         [ 0.2081, -0.3069,  0.1994,  0.1640, -0.3619],
         [ 0.2081, -0.3069,  0.1994,  0.1640, -0.3619]],

        [[ 0.2169, -0.2792,  0.1324,  0.1725, -0.3363],
         [ 0.2146, -0.2847,  0.0831,  0.1790, -0.3274],
         [ 0.1729, -0.2822,  0.0625,  0.1681, -0.3329],
         [ 0.1447, -0.2657,  0.1248,  0.1566, -0.3573],
         [ 0.1008, -0.2979,  0.1863,  0.1518, -0.4057]]],
       grad_fn=<ViewBackward0>)

In [70]:
nn.LogSoftmax()(linear(output[~pad_mask]))

/home/yuntae_kang/.pyenv/versions/3.12.5/envs/imb_data/lib/python3.12/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([[-1.4011, -1.8972, -1.4855, -1.4455, -1.9543],
        [-1.3939, -1.8932, -1.5253, -1.4295, -1.9358],
        [-1.4178, -1.8729, -1.5282, -1.4226, -1.9236],
        [-1.4011, -1.8972, -1.4855, -1.4455, -1.9543],
        [-1.3939, -1.8932, -1.5253, -1.4295, -1.9358],
        [-1.4011, -1.8972, -1.4855, -1.4455, -1.9543],
        [-1.3939, -1.8932, -1.5253, -1.4295, -1.9358],
        [-1.4178, -1.8729, -1.5282, -1.4226, -1.9236],
        [-1.4495, -1.8599, -1.4694, -1.4376, -1.9515],
        [-1.4851, -1.8838, -1.3996, -1.4341, -1.9916]],
       grad_fn=<LogSoftmaxBackward0>)

In [114]:
output = nn.functional.log_softmax(linear(output[~pad_mask]), dim=-1)

In [129]:
output

tensor([[-1.4144, -1.9246, -1.4312, -1.4520, -1.9817],
        [-1.4154, -1.9225, -1.4356, -1.4473, -1.9824],
        [-1.4165, -1.9175, -1.4442, -1.4410, -1.9816],
        [-1.4144, -1.9246, -1.4312, -1.4520, -1.9817],
        [-1.4154, -1.9225, -1.4356, -1.4473, -1.9824],
        [-1.4144, -1.9246, -1.4312, -1.4520, -1.9817],
        [-1.4154, -1.9225, -1.4356, -1.4473, -1.9824],
        [-1.4165, -1.9175, -1.4442, -1.4410, -1.9816],
        [-1.4158, -1.9159, -1.4443, -1.4435, -1.9800],
        [-1.4165, -1.9159, -1.4405, -1.4451, -1.9829]],
       grad_fn=<LogSoftmaxBackward0>)

In [164]:
torch.exp(output).multinomial(1)

tensor([[3],
        [2],
        [0],
        [1],
        [0],
        [1],
        [3],
        [1],
        [3],
        [1]])

In [103]:
a[:, 0].shape

torch.Size([3])

In [113]:
a[:, [0, 1]]

tensor([[1, 2],
        [1, 2],
        [1, 2]])

In [105]:
a.chunk(a.size(1), dim=-1)[0].shape

torch.Size([3, 1])

In [71]:
a.device

device(type='cpu')

In [75]:
param = next(linear.parameters())

In [94]:
param.data

tensor([[-0.0194,  0.0170,  0.0212,  0.0014],
        [ 0.0200,  0.0238,  0.0315,  0.0009],
        [-0.0230,  0.0378, -0.0244, -0.0325],
        [ 0.0396,  0.0358,  0.0336, -0.0331],
        [-0.0322,  0.0056,  0.0405,  0.0256]])

In [93]:
param.data.uniform_(-0.05, 0.05)

tensor([[-0.0194,  0.0170,  0.0212,  0.0014],
        [ 0.0200,  0.0238,  0.0315,  0.0009],
        [-0.0230,  0.0378, -0.0244, -0.0325],
        [ 0.0396,  0.0358,  0.0336, -0.0331],
        [-0.0322,  0.0056,  0.0405,  0.0256]])